<a href="https://colab.research.google.com/github/Vishnu-919/CS-Assignment2/blob/main/CS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install whois requests beautifulsoup4 scikit-learn pandas numpy matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import requests
import whois
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('/content/dataset.csv')

# Inspect first rows
df.head(), df.shape


(   index  having_IPhaving_IP_Address  URLURL_Length  Shortining_Service  \
 0      1                          -1              1                   1   
 1      2                           1              1                   1   
 2      3                           1              0                   1   
 3      4                           1              0                   1   
 4      5                           1              0                  -1   
 
    having_At_Symbol  double_slash_redirecting  Prefix_Suffix  \
 0                 1                        -1             -1   
 1                 1                         1             -1   
 2                 1                         1             -1   
 3                 1                         1             -1   
 4                 1                         1             -1   
 
    having_Sub_Domain  SSLfinal_State  Domain_registeration_length  ...  \
 0                 -1              -1                           -1  ...   


In [ ]:
print("Columns:", df.columns.tolist())
print("Unique labels:", df['Result'].unique())  # may be “class” or “label”

Columns: ['index', 'having_IPhaving_IP_Address', 'URLURL_Length', 'Shortining_Service', 'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix', 'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length', 'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor', 'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL', 'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe', 'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank', 'Google_Index', 'Links_pointing_to_page', 'Statistical_report', 'Result']
Unique labels: [-1  1]


In [ ]:
def extract_url_features(url):
    features = {}
    features['url_length'] = len(url)
    features['num_digits'] = sum(c.isdigit() for c in url)
    features['num_dots'] = url.count('.')
    features['num_hyphens'] = url.count('-')
    features['has_at'] = int('@' in url)
    features['https'] = int(url.lower().startswith('https'))
    features['num_subdirs'] = url.count('/')
    # entropy
    freq = [url.count(c)/len(url) for c in set(url)]
    features['entropy'] = -sum([p * np.log2(p) for p in freq if p > 0])
    return features

def extract_html_features(url):
    try:
        resp = requests.get(url, timeout=5)
        soup = BeautifulSoup(resp.text, 'html.parser')
        text = soup.get_text()
        num_iframes = len(soup.find_all('iframe'))
        num_forms = len(soup.find_all('form'))
        num_scripts = len(soup.find_all('script'))
    except Exception as e:
        text = ""
        num_iframes = 0
        num_forms = 0
        num_scripts = 0
    return {
        'text': text,
        'num_iframes': num_iframes,
        'num_forms': num_forms,
        'num_scripts': num_scripts
    }


In [ ]:
records = []
numeric_cols = ['url_length','num_digits','num_dots','num_hyphens',
                'has_at','https','num_subdirs','entropy',
                'num_iframes','num_forms','num_scripts']

for idx, row in df.iterrows():
    url = row['url']  # or whichever column holds URL
    label = row['Result']
    url_feats = extract_url_features(url)
    html_feats = extract_html_features(url)
    rec = {**url_feats, **html_feats, 'Result': label}
    records.append(rec)

feat_df = pd.DataFrame(records)
feat_df.head(), feat_df.shape


KeyError: 'url'

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
X_text = tfidf.fit_transform(feat_df['text'])

svd = TruncatedSVD(n_components=50)
X_text_red = svd.fit_transform(X_text)

X_num = feat_df[numeric_cols].fillna(0).values
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

X = np.hstack([X_num_scaled, X_text_red])
y = feat_df['Result'].values


NameError: name 'feat_df' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(64,32), max_iter=300, random_state=42)

rf.fit(X_train, y_train)
mlp.fit(X_train, y_train)


In [ ]:
rf_pred = rf.predict_proba(X_train)[:,1]
mlp_pred = mlp.predict_proba(X_train)[:,1]

meta_X = np.vstack([rf_pred, mlp_pred]).T
meta_model = LogisticRegression().fit(meta_X, y_train)

rf_test = rf.predict_proba(X_test)[:,1]
mlp_test = mlp.predict_proba(X_test)[:,1]
meta_test = np.vstack([rf_test, mlp_test]).T

final_pred = (meta_model.predict_proba(meta_test)[:,1] > 0.5).astype(int)


In [ ]:
print("Accuracy:", accuracy_score(y_test, final_pred))
print("\nClassification Report:\n", classification_report(y_test, final_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, final_pred))

In [ ]:
import matplotlib.pyplot as plt

importances = rf.feature_importances_[:len(numeric_cols)]
plt.figure(figsize=(8,5))
plt.barh(numeric_cols, importances)
plt.xlabel("Importance")
plt.title("Feature Importances (URL + HTML numeric features)")
plt.show()


In [ ]:
new_url = "http://secure-login-example-update.com"
url_feats = extract_url_features(new_url)
html_feats = extract_html_features(new_url)
rec = {**url_feats, **html_feats}
num_part = scaler.transform([[rec[c] for c in numeric_cols]])
text_vec = tfidf.transform([rec['text']])
text_red = svd.transform(text_vec)
X_new = np.hstack([num_part, text_red])

rf_p = rf.predict_proba(X_new)[:,1]
mlp_p = mlp.predict_proba(X_new)[:,1]
meta_p = np.vstack([rf_p, mlp_p]).T
pred_label = (meta_model.predict_proba(meta_p)[:,1] > 0.5).astype(int)[0]

print("URL:", new_url)
print("Prediction:", "Phishing" if pred_label == 1 else "Legitimate")
